In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

In [ ]:
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station


In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)


# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
most_recent = session.query(Measurement.date).order_by(Measurement.date.desc()).first().date
most_recent

In [ ]:
one_year_stop = dt.datetime.strptime(most_recent, '%Y-%m-%d') - dt.timedelta(days=365)
one_year_stop

In [ ]:
dates_precip = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= one_year_stop).all()
dates_precip

In [ ]:
precip_df = pd.DataFrame(dates_precip, columns=['Date', 'Precipitation'])
precip_df = precip_df.set_index('Date').dropna()
precip_df.head()

In [ ]:
precip_df.plot()


In [ ]:
precip_df.describe()

In [ ]:
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.


# Perform a query to retrieve the data and precipitation scores


# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset

### print query of Station to pull out translated SQL code for the query to find column names ###
    ### print(session.query(Station)) ###
    
print(session.query(Station))

In [ ]:
print(session.query(Measurement))

In [ ]:
measure_data = session.query(Measurement.station, Measurement.date, Measurement.prcp, Measurement.tobs)
measure_df = pd.DataFrame(measure_data, columns = ['station', 'date', 'precipitation', 'tobs'])
measure_df.set_index(['station'])
measure_df.dropna()

measure_df

In [ ]:
#total stations using measure data
total_stations = len(measure_df['station'].unique())
print(f'There are {total_stations} total stations.')

In [ ]:
station_data = session.query(Station.id, Station.station, Station.name, Station.latitude, Station.longitude, Station.elevation).all()
station_data_df = pd.DataFrame(station_data, columns=['id', 'station', 'name', 'latitude', 'longitude', 'elevation'])
station_data_df

In [ ]:
# confirming total stations using station data df
len(station_data_df['station'].unique())

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
most_active = session.query(Measurement.station, func.count(Measurement.station)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.station).desc()).all()

most_active

In [ ]:
#most active station using dataframe.  note that totals agree with above query.
measure_df['station'].value_counts()

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active_station_id = most_active[0][0]
most_active_station_id

session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
    filter(Measurement.station == most_active_station_id).all()

In [ ]:
#find the most recent temperature data for most active station id
most_active_most_recent = session.query(Measurement.date).filter(Measurement.station == most_active_station_id).\
    order_by(Measurement.date.desc()).first().date
most_active_most_recent


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station
last12_temp_mostactive = dt.datetime.strptime(most_active_most_recent, '%Y-%m-%d') - dt.timedelta(days=365)
last12_temp_mostactive

temp_data = session.query(Measurement.date, Measurement.tobs).filter(Measurement.date >= last12_temp_mostactive).all()
temp_data

In [ ]:
# plot the results as a histogram
temp_data_df = pd.DataFrame(temp_data, columns = ('Date', 'Temperature (F)'))
temp_data_df = temp_data_df
#(54.0, 85.0, 71.66378066378067)

#maxtemp temp_data_df['Temperature']

temps = temp_data_df['Temperature (F)']

In [ ]:

plt.hist(temps, bins=12, color='b')
plt.title("Temperature Recordings from 08/2016 - 08/2017 for Waihee Station, HI US", fontweight='bold',fontsize=20)
plt.ylabel('Frequency', weight='bold', fontsize=15)
plt.xlabel('Temperature (F)', weight='bold', fontsize=15)
plt.show()

# Close session

In [ ]:
# Close Session
session.close()